In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
towns = gpd.read_file('C:\Code\PopFrame\examples/towns.geojson')
towns = towns.drop(columns=['population'])
towns = towns.rename(columns={'rounded_population': 'population'})
# adj_mx = pd.read_pickle('data/acc_mx.pickle')
# adj_mx.index = towns.index
# adj_mx.columns = towns.index

In [3]:
towns

,name,name_norm,adm_unit_id,modeled_population,population,is_modeled,geometry
0,Болото,БОЛОТО,0,9.616352,10.0,True,POINT (33.75892 59.36226)
1,Большой Остров,БОЛЬШОЙ ОСТРОВ,0,68.000000,68.0,False,POINT (33.78600 59.47517)
2,Бор,БОР,0,1734.000000,1734.0,False,POINT (33.79236 59.47356)
3,Бороватое,БОРОВАТОЕ,0,9.616352,10.0,True,POINT (33.77572 59.44249)
4,Бочево,БОЧЕВО,0,9.616352,10.0,True,POINT (33.67728 59.32819)
...,...,...,...,...,...,...,...
2925,Апраксин Бор,АПРАКСИН БОР,191,312.952405,313.0,True,POINT (31.23421 59.17021)
2926,Александровка,АЛЕКСАНДРОВКА,191,312.952405,313.0,True,POINT (31.31924 59.18702)
2927,Большая Горка,БОЛЬШАЯ ГОРКА,191,312.952405,313.0,True,POINT (31.47462 59.29408)
2928,Дроздово,ДРОЗДОВО,191,5.000000,5.0,False,POINT (31.46265 59.30308)


In [4]:
import requests
async def get_accessibility_matrix(region_id : int, graph_type : str) -> pd.DataFrame:
        res = requests.get('http://10.32.1.65:5700' + f'/api_v1/{region_id}/get_matrix', {
          'graph_type': graph_type
        })
        json = res.json()
        return pd.DataFrame(json['values'], index=json['index'], columns=json['columns'])

adj_mx = await get_accessibility_matrix(1, 'drive')
adj_mx

,207,208,209,210,211,212,213,214,215,216,...,3128,3129,3130,3131,3132,3133,3134,3135,3136,3137
207,0.000,11.687,12.024,10.794,7.768,3.431,21.873,29.029,8.415,17.122,...,175.052,167.133,165.000,170.273,195.276,195.276,188.976,172.755,171.412,183.300
208,11.687,0.000,0.337,4.313,18.027,13.690,29.914,37.070,16.456,10.347,...,163.986,156.067,153.934,159.207,184.210,184.210,177.910,161.689,160.346,172.234
209,12.024,0.337,0.000,4.650,18.364,14.027,30.251,37.407,16.248,10.684,...,163.649,155.730,153.597,158.870,183.873,183.873,177.573,161.352,160.009,171.897
210,10.794,4.313,4.650,0.000,17.134,12.797,29.021,36.177,15.563,9.748,...,167.678,159.759,157.626,162.899,187.902,187.902,181.602,165.381,164.038,175.926
211,7.768,18.027,18.364,17.134,0.000,4.337,28.213,35.369,14.755,23.462,...,181.392,173.473,171.340,176.613,201.616,201.616,195.316,179.095,177.752,189.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,195.417,184.351,184.014,188.043,201.757,197.420,213.644,220.800,197.759,194.077,...,24.994,29.115,31.820,32.255,0.000,0.000,17.446,39.575,38.232,32.092
3134,189.116,178.050,177.713,181.742,195.456,191.119,207.343,214.499,191.458,187.776,...,17.942,22.063,24.768,25.203,17.446,17.446,0.000,32.523,31.180,25.040
3135,172.899,161.833,161.496,165.525,179.239,174.902,191.126,198.282,175.241,171.559,...,18.603,10.684,7.755,13.824,39.579,39.579,32.527,0.000,1.343,26.851
3136,171.556,160.490,160.153,164.182,177.896,173.559,189.783,196.939,173.898,170.216,...,17.260,9.341,6.412,12.481,38.236,38.236,31.184,1.343,0.000,25.508


In [5]:
adj_mx.index = towns.index
adj_mx.columns = towns.index

In [6]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (33.75892 59.36226),Болото,10,Малое сельское поселение
1,POINT (33.78600 59.47517),Большой Остров,68,Малое сельское поселение
2,POINT (33.79236 59.47356),Бор,1734,Большое сельское поселение
3,POINT (33.77572 59.44249),Бороватое,10,Малое сельское поселение
4,POINT (33.67728 59.32819),Бочево,10,Малое сельское поселение
...,...,...,...,...
2925,POINT (31.23421 59.17021),Апраксин Бор,313,Среднее сельское поселение
2926,POINT (31.31924 59.18702),Александровка,313,Среднее сельское поселение
2927,POINT (31.47462 59.29408),Большая Горка,313,Среднее сельское поселение
2928,POINT (31.46265 59.30308),Дроздово,5,Малое сельское поселение


In [10]:
crs = towns.estimate_utm_crs()
region = region.to_crs(crs)
towns = towns.to_crs(crs)

In [11]:
from popframe.method.popuation_frame import PopulationFrame
from popframe.models.region import Region

region_model = Region(
  region = region,
  towns=towns,
  accessibility_matrix=adj_mx
)
# region.to_pickle('Ленинградская область.pickle')
# region_model = Region.from_pickle('Ленинградская область.pickle')

In [7]:
from popframe.method.popuation_frame import PopulationFrame
from popframe.models.region import Region
# region_model = Region.from_pickle('C:\Code\PopFrame\Ленинградская область.pickle')
region_model = Region.from_pickle('C:\Code\PopFrame\Тюменская область.pickle')

In [12]:
region_model.accessibility_matrix

,4909,4910,4911,4912,4913,4914,4915,4916,4917,4918,...,5178,5179,5180,5181,5182,5183,5184,5185,5186,5187
4909,0.000,22.150,36.102,38.787,49.776,74.585,74.483,40.174,42.004,55.608,...,315.540,323.348,342.385,383.493,111.044,69.270,301.159,262.506,211.399,198.477
4910,22.150,0.000,58.252,60.937,71.926,96.735,96.633,62.324,64.154,77.758,...,337.690,345.498,347.253,388.361,133.194,91.420,306.027,284.656,233.549,220.627
4911,36.087,58.237,0.000,29.645,18.986,52.431,52.329,17.906,36.237,33.454,...,306.398,314.206,334.436,381.230,101.902,60.128,314.665,253.364,202.257,189.335
4912,38.787,60.937,29.645,0.000,43.319,68.128,68.026,33.717,19.565,49.151,...,280.331,288.139,308.369,355.163,75.835,34.061,288.598,227.297,176.190,163.268
4913,49.761,71.911,18.986,43.319,0.000,54.821,54.719,31.580,49.911,35.844,...,320.072,327.880,348.110,394.904,115.576,73.802,328.339,267.038,215.931,203.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5183,69.326,91.476,60.184,34.117,73.858,53.025,77.247,64.256,50.104,74.162,...,257.891,265.699,285.929,332.723,53.395,0.000,266.158,204.857,153.750,140.828
5184,301.169,306.037,314.775,288.708,328.449,313.547,337.769,318.847,304.695,334.281,...,105.626,127.183,66.036,103.905,212.873,266.201,0.000,165.376,180.665,205.423
5185,262.703,284.853,253.561,227.494,267.235,252.333,276.555,257.633,243.481,273.067,...,106.433,114.241,121.716,168.510,151.659,204.987,165.504,0.000,57.856,90.047
5186,212.097,234.247,202.955,176.888,216.629,201.727,225.949,207.027,192.875,222.461,...,105.596,113.404,133.634,180.428,101.053,154.381,177.422,57.653,0.000,39.441


In [11]:
towns = region_model.get_towns_gdf()
towns

,id,name,population,level,geometry
4909,4909,Ощепковское,2486594,Крупнейший город,POINT (599483.006 6259295.831)
4910,4910,Назаровское,1936749,Крупнейший город,POINT (617753.459 6268896.015)
4911,4911,Шевыринское,2333260,Крупнейший город,POINT (595421.530 6227657.436)
4912,4912,Тушнолобовское,2748389,Крупнейший город,POINT (569111.118 6227703.994)
4913,4913,Партизанское,1774097,Крупнейший город,POINT (611450.392 6216696.039)
...,...,...,...,...,...
5183,5183,Ишим,541658,Крупный город,POINT (528987.182 6216792.710)
5184,5184,Тобольск,2845195,Крупнейший город,POINT (462957.642 6452342.842)
5185,5185,Тюмень,498439,Крупный город,POINT (289728.651 6339681.922)
5186,5186,Ялуторовск,2057265,Крупнейший город,POINT (334272.821 6281579.091)


In [13]:
frame_method = PopulationFrame(region=region_model)
G = frame_method.build_network_frame()
num_edges = G.number_of_edges()
print("Количество рёбер в графе:", num_edges)
frame_method.get_graph_html(G, 'tumen.html')

Количество рёбер в графе: 278


In [3]:
frame_method = PopulationFrame(region=region_model)
G = frame_method.build_network_frame()
num_edges = G.number_of_edges()
print("Количество рёбер в графе:", num_edges)
output = frame_method.build_circle_frame(output_type='gdf') 
output

Количество рёбер в графе: 2929


,id,name,population,level,geometry
207,207,Болото,2950210,Крупнейший город,"POLYGON ((3777146.521 8259101.277, 3777054.450..."
208,208,Большой Остров,2943043,Крупнейший город,"POLYGON ((3780137.257 8283806.182, 3780045.298..."
209,209,Бор,446748,Крупный город,"POLYGON ((3769188.510 8283452.668, 3769152.682..."
210,210,Бороватое,678585,Крупный город,"POLYGON ((3769066.074 8276646.700, 3769021.917..."
211,211,Бочево,2498355,Крупнейший город,"POLYGON ((3766533.437 8251662.279, 3766448.710..."
...,...,...,...,...,...
3133,3133,Апраксин Бор,1325159,Крупнейший город,"POLYGON ((3489790.504 8217266.893, 3489728.798..."
3134,3134,Александровка,968051,Крупный город,"POLYGON ((3497394.615 8220918.561, 3497341.875..."
3135,3135,Большая Горка,2910847,Крупнейший город,"POLYGON ((3522731.206 8244221.253, 3522639.752..."
3136,3136,Дроздово,1353209,Крупнейший город,"POLYGON ((3515355.513 8246184.181, 3515293.158..."


In [4]:
from popframe.method.aglomeration import AgglomerationAnalysis

agglomeration = AgglomerationAnalysis(region=region_model)
agglomerations_gdf = agglomeration.build_agglomeration_voronoi(G)


In [5]:
frame_method.get_graph_html(G, 'final_graph_with_legend.html')

In [6]:
m = agglomerations_gdf.explore()
towns.explore(m=m)

NameError: name 'towns' is not defined

In [17]:
from popframe.method.territory_evaluation import TerritoryEvaluation
from popframe.models.region import Region
from shapely.geometry import shape


geojson_data = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [29.982879431084967, 59.363554752593245],
                [29.982879431084967, 59.322083801173534],
                [30.109075699649765, 59.322083801173534],
                [30.109075699649765, 59.363554752593245],
                [29.982879431084967, 59.363554752593245]
            ]
        ]
    },
    "properties": {},
    "type": "Feature"
}

# Преобразуем словарь в геометрию
geom = shape(geojson_data['geometry'])

# Создаем GeoDataFrame
territories_gdf = gpd.GeoDataFrame([{'geometry': geom}], crs='EPSG:4326')

evaluation = TerritoryEvaluation(region=region_model)
# Пример вызова функции
results = evaluation.population_criterion(territories_gdf=territories_gdf)
results

[{'project': None,
  'average_population_density': 17.2,
  'total_population': 39739,
  'score': 4.0,
  'interpretation': 'Территория имеет хорошие показатели численности и плотности населения, что способствует ее активному развитию.'}]

In [18]:
from popframe.method.territory_evaluation import TerritoryEvaluation
from shapely.geometry import shape


geojson_data = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [29.982879431084967, 59.363554752593245],
                [29.982879431084967, 59.322083801173534],
                [30.109075699649765, 59.322083801173534],
                [30.109075699649765, 59.363554752593245],
                [29.982879431084967, 59.363554752593245]
            ]
        ]
    },
    "properties": {},
    "type": "Feature"
}

geom = shape(geojson_data['geometry'])

# Создаем GeoDataFrame
territories_gdf = gpd.GeoDataFrame([{'geometry': geom}], crs='EPSG:4326')

result = evaluation.evaluate_territory_location(territories_gdf=territories_gdf)
result

[{'territory': 'None',
  'score': 5,
  'interpretation': 'Территория находится внутри или непосредственной близости населенного пункта уровня Малый город',
  'closest_settlement': 'Сиверский',
  'closest_settlement1': None,
  'closest_settlement2': None}]